# A Set-Covering Approach to Minimize the Variety of Standard Chemical Process Pumps in Equipment Pools

## Abstract

A decision framework is presented to minimize the number of distinct ISO 2858 compliant standard chemical process pumps in an equipment pool. 
Two model variants are introduced: (i) the set cover problem to find the minimum number of ISO-sizes required to meet a given set of specifications, and (ii) the maximum coverage problem to allow for a trade-off to be made between application coverage and equipment variety in the pump pool. 
The decision-models are applied to a data set containing specifications for pumping applications provided by Evonik Operations GmbH. 
A kernel density estimator is trained using that data to provide a non-parametric generative model that is used to anticipate likely future specifications for which a pump pool is to be set up. 
The decision framework is applied for a generated sample of 3268 applications choosing from 68 available ISO sizes. 
The results suggest, that equipment pools comprising just a subset of all available ISO-sizes may be sufficient to cover a broad range of applications. 
Specifically, in our case study 36 out of 68 standard pumps cover 95% of all applications suitable for standard chemical process pumps; which is 62% of the total specifications sample generated. 
Numerically, we found the model to scale well with overall problem size. 
The entire data processing pipeline including all pre-processing of data can be executed in the order of a few seconds on standard laptop computer hardware, making the approach computational tractable even for large scale industrial analyses. 
By including an established model of the centrifugal pump operating ranges taken from a dimensioning tool developed at German chemical industry association VCI, we eliminate the need for further specialized software and minimize equipment-data requirements. 
Thus, we provide a simple, clear and concise model that equipment managers can readily integrate in their workflows.

In [ ]:
import os
from datetime import datetime
from pathlib import Path

DATA_DIR = Path(r"./data/")
OUT_DIR = Path(r"./{}_results/".format(datetime.today().strftime('%y%m%d')))

if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)

## PiC LF03: Guideline on centrifugal pump units

Following the VCI (German chemical industry association) guideline on centrifugal pump units (PiC LF03), maximum rages for continuous operation can be estimated for each ISO standard size. 
Within this guide size-spezific and speed-specific ranges for contiuous operation (useful hydraulic ranges) are defined through four simultaneous linear inequalities; the intersects of which can be obtained for each size and speed designation from a supplementary *Microsoft Execl Dimensioning Tool*. 
The dimensioning tool is available for download from the [VCI website](https://www.vci.de/services/leitfaeden/kreiselpumpenaggregate-lf03-anwendung-bewertung-strategien-gemaess-din-en-iso-2858-vci-leitfaden.jsp). 
Subsequent steps within our framework require the location of those intersects given as concrete numerical values. 
A unitilty command line script `init.py` extracts the data given within the dimensioning tool and writes them into a `csv`-file.   

#### Exemplary usage
```shell
python -m scripts.init -i data/2020-08-05-lf03-anhang-a-berechnungsblatt.xls
```

In [ ]:
!python -m scripts.init --help

### File contents

In [ ]:
import pandas as pd

iso = pd.read_csv(
    DATA_DIR / "2020-08-05-lf03-anhang-a-berechnungsblatt.csv",
    sep=",",
    index_col="register"
) 
iso.head()

### Visualization of *useful hydraulic ranges*

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from scripts.plot import _draw_reverse_engineered_region

In [ ]:
dropdown = widgets.Dropdown(
    options=iso.index.to_list(),
    value='R01',
    description='register:',
    disabled=False,
)
output = widgets.Output()

def on_value_change(change):
    with output:
        fig = plt.figure()
        ax = fig.gca()
        _draw_reverse_engineered_region(iso, change["new"], ax, color='black', linestyle='-',marker='None')
        _ = ax.set_xlabel("FLOW RATE $Q$ in $\mathrm{m}^3 / \mathrm{h}$")
        _ = ax.set_ylabel("HEAD $H$ in $\mathrm{m}$")
        ax.spines[['right', 'top']].set_visible(False)
        output.clear_output()
        plt.show()

dropdown.observe(on_value_change, names='value')

In [ ]:
display(dropdown,output)

In [ ]:
from scripts.plot import gen_fig_ranges_overview

In [ ]:
fig = gen_fig_ranges_overview()

## Specifications sample

Given a known design point and an ISO standard size it can be determined whether the pump unit can cover that application or not. 
In order to minimize pool variety we wish to solve the set cover and maximum coverage problems respectively for a list of applications. 
This list can be given by a known set of specifications or obtained by sampling from a generative model. 
In our case study, we followed the latter approach. 

### Kernel density estimation

A kernel density estimator (KDE) is trained from a set of given specifications. 
The KDE provides a non-parametric model from which new specifications can be generated. 
This allows for a larger sample to be treated than with the original data. 
The command line script `sample.py` generates such a sample from a tabulated set of specifications, that is flow rate, head and kin. viscosity tuples.  

#### Exemplary usage
```shell
python scripts/sample.py -m odbc -c ./.config/pyodbc.yaml -q ./.config/query.sql  -n 5000
```

In [ ]:
!python -m scripts.sample --help

**Note**: In this notebook, we did not generate a new sample but reused the existing sample from the `results` directory in order to reproduce the rest of the results deterministically. 

In [ ]:
data_sampled = pd.read_csv(Path(r"results/data_231009_sample.csv"), index_col="element_id")
data_sampled.head()

### Preprocessing

Solving both the set cover and the maximum coverage problem requires providing the 0/1-values of the characteristic vectors $\{a_{ij}\}_{i=1}^N$ of sets $S_j$, i.e. which applications fall within the ranges of which pump size designation $j = 1,\dots,M$. 
This is done using the `preprocess.py` script. 
It writes a *Pyomo Data Command File* for the set cover problem given a list of flow rate, head and viscosity specifications $j$. 
Specifications that fall outside any of the useful hydraulic ranges or lie outside a viscosity range of $\nu < 10 \, \mathrm{cS}$ are rejected. 
Accepted and rejected parts of the sample are documented in a `csv`-file `<path-to>/data_yymmdd_elements.csv`, where `yymmdd` is the date format of the date of execution. 
If `data_yymmdd_elements.csv` already exists it is read and not overwritten. 
The output directory `<path-to>` is taken from the `OUTPUT` argument (directory of the `.dat`-file) specified.

#### Exemplary usage
```shell
python -m scripts.preprocess -i ./results/data_231009_sample.csv -o ./out/data_231121_set_cover.dat
```

In [ ]:
sample = Path(r"./results/data_231009_sample.csv")
out = OUT_DIR / "data_{}_set_cover.dat".format(datetime.today().strftime('%y%m%d'))

!python -m scripts.preprocess -i {sample} -o {out}

In [ ]:
fname = OUT_DIR / "data_{}_elements.csv".format(datetime.today().strftime('%y%m%d'))
data_sampled = pd.read_csv(fname, index_col="element_id")
elements = data_sampled[data_sampled["mask_total"]]
elements.head()

In [ ]:
print("total sample: {}\nwithin the viscosity constraints: {}\nwithin any `useful hydraulic range`: {}\naccepted: {}".format(
        data_sampled.shape[0],
        data_sampled["mask_viscosity"].sum(),
        data_sampled["mask_coverage"].sum(),
        data_sampled["mask_total"].sum()
    )
)

### Visualization of sample data

In [ ]:
from scripts.plot import gen_fig_sample

fig = gen_fig_sample(
        fname_elements=OUT_DIR / "data_{}_elements.csv".format(datetime.today().strftime('%y%m%d'))
    )

## Set Cover Problem

$$
\begin{aligned}
    \min            &\quad \sum_{i = 1}^{M} x_i \\
    \mathrm{s.t.}   &\quad \sum_{j = 1}^{M} a_{kj} x_j \geq 1 \qquad \forall k = 1,\ldots,N\\
                    &\quad x_i \in \{0,1\} \qquad \forall i = 1,\ldots,M
\end{aligned}
$$

#### Exemplary usage

The set cover problem is solved calling the `pyomo` command from the command line (see [Pyomo Documentation](https://pyomo.readthedocs.io/en/stable/working_abstractmodels/pyomo_command.html)).  
```shell
pyomo solve --solver=gurobi --save-results=out/results_example_set_cover.yaml model_set_cover.py data/example_set_cover.dat
```

In [ ]:
data = OUT_DIR / "data_{}_set_cover.dat".format(datetime.today().strftime('%y%m%d'))
out = OUT_DIR / "results_{}_set_cover.yaml".format(datetime.today().strftime('%y%m%d'))

!pyomo solve --solver=gurobi --save-results {out} model_set_cover.py {data}

## Maximum Coverage Problem

$$
\begin{aligned}
    \max            &\quad z = \sum_{i = 1}^{N} y_i \\
    \mathrm{s.t.}   &\quad \sum_{j = 1}^{M} a_{kj} x_j \geq y_k \qquad \forall k = 1,\ldots,N\\
                    &\quad \sum_{j = 1}^{M} x_j \leq K \\\
                    &\quad x_i \in \{0,1\} \qquad \forall i = 1,\ldots,M \\
                    &\quad y_i \in \{0,1\} \qquad \forall i = 1,\ldots,N \\
\end{aligned}
$$

#### Exemplary usage

The maximum coverage problem, similarly, is also solved calling the `pyomo` command from the command line (see [Pyomo Documentation](https://pyomo.readthedocs.io/en/stable/working_abstractmodels/pyomo_command.html)).  

Solving the maximum coverage problem requires specifying a value for the parameter $K$ (`max_num_sets`) used in formulating the cardinality constraint. 
All other data can be imported from the *data command file* written to solve the set cover problem. 

In [ ]:
for k in [10,25,50]:
    fname = OUT_DIR / "data_{}_maximum_coverage_{}.dat".format(datetime.today().strftime('%y%m%d'),k)
    with open(fname,"w+") as f:
        f.write("include \"{}/data_{}_set_cover.dat\";\n".format(OUT_DIR.as_posix(),datetime.today().strftime('%y%m%d')))
        f.write("param max_num_sets := {};\n".format(k))

The problem is then solved by calling, for instance,
```shell
pyomo solve --solver=gurobi --save-results=out/results_example_maximum_coverage.yaml model_maximum_coverage.py data/example_maximum_coverage.dat
```

In [ ]:
data = OUT_DIR / "data_{}_maximum_coverage_25.dat".format(datetime.today().strftime('%y%m%d'))
out = OUT_DIR / "results_{}_maximum_coverage_25.yaml".format(datetime.today().strftime('%y%m%d'))

!pyomo solve --solver=gurobi --save-results {out} model_maximum_coverage.py {data}

#### Visualization of solutions to the maximum coverage problem

In [ ]:
from scripts.plot import gen_fig_solution

In [ ]:
fig = gen_fig_solution(
    fname_sol=OUT_DIR/"results_{}_maximum_coverage_25.yaml".format(datetime.today().strftime('%y%m%d')),
    fname_elements=OUT_DIR/"data_{}_elements.csv".format(datetime.today().strftime('%y%m%d'))
)

### Pareto-Plot

Solving the *Maximium Coverage Problem* for every integer value $0 < K < \bar{K}$, where $\bar{K}$ is the optimal solution of the *Set Cover Problem*, gives a Pareto frontier. 
The utility script `pareto.py` generate such Pareto frontier. 
It first solves the *Set Cover Problem* to bound the parameter $K$ above ($\bar{K}$) and thereafter creates and solves instances of the *Maximium Coverage Problem* for every $0 < K < \bar{K}$. 

#### Exemplary usage 
```shell
python -m scripts/pareto.py -i results/data_231009_set_cover.dat -o results/results_231009_pareto_frontier.csv --solver gurobi
```

In [ ]:
data = OUT_DIR / "data_{}_set_cover.dat".format(datetime.today().strftime('%y%m%d'))
out = OUT_DIR / "results_{}_pareto_frontier.csv".format(datetime.today().strftime('%y%m%d'))

!python -m scripts.pareto -i {data} -o {out} --solver gurobi

In [ ]:
from scripts.plot import gen_pareto_plot
fig = gen_pareto_plot(fname=out)

## Plots shown in the article

The final Python script `plot.py` contains code to generate all plots presented within our article. If run as a command line application `plot.py` generates all plots from the results files and saves them in the specified format. 
To ensure the files are saved provide the `-s` flag. 
Without it, the plots are generated but not saved (allows for a dry run). 
All plots are generated using `matplotlib`; the format option is passed to `matplotlib.pyplot.savefig`. 

### Exemplary usage 
```shell
python -m scripts.plot -s -d figs -f jpg
```

In [ ]:
config = {
        "ranges_overview":"2020-08-05-lf03-anhang-a-berechnungsblatt.csv",
        "sample": {
            "fname_elements":"results/data_231009_elements.csv"
        },
        "solution":{
            "fname_sol":"results/results_231009_maximum_coverage_25.yaml",
            "fname_elements":"results/data_231009_elements.csv"
        },
        "pareto":"results/results_231009_pareto_frontier.csv"
    }

In [ ]:
import yaml

fname = OUT_DIR/"config_{}_plots.yaml".format(datetime.today().strftime('%y%m%d'))

with open(fname, "w+") as f:
    f.write(yaml.safe_dump(config))

In [ ]:
!python -m scripts.plot -s -d {OUT_DIR} -f jpg --config {fname}

## Computational Study

In [ ]:
import os
import yaml
import pandas as pd

def benchmark(model, data, *solvers, **kwargs):

    n = kwargs.get("n", 10)
    tmp = kwargs.get("tmp", "tmp.yaml")
    out = kwargs.get("out", None)
    save = kwargs.get("save", False)
    
    times = pd.DataFrame(index=range(n), columns=solvers)
    if save:
        if out is None:
            raise IOError("provide file name to write")
    
    for idx in times.index:
        for solver in times.columns:
            os.system(f"pyomo solve --solver={solver} --save-results={tmp} {model} {data}")
            with open(tmp,'r') as f:
                results = yaml.safe_load(f)
            
            if (results["Solver"][0]["Status"] == "ok"):
                times.iloc[idx][solver] = results["Solver"][0]["Time"]

    os.remove(tmp)
    
    if save:
        times.to_csv(out, index=False)

    return times

### Benchmarks

In [ ]:
if False:
    data = OUT_DIR / "data_{}_set_cover.dat".format(datetime.today().strftime('%y%m%d'))
    out = OUT_DIR / "results_{}_study_solver_time_sc.csv".format(datetime.today().strftime('%y%m%d'))
    
    sc = benchmark("model_set_cover.py", data, "cplex", "glpk", "gurobi", n=50, save=True, out=out)
else:
    sc = pd.read_csv(r"results/results_231012_study_solver_time_sc.csv")

sc.quantile()

In [ ]:
if False:
    data = OUT_DIR / "data_{}_maximum_coverage_10.dat".format(datetime.today().strftime('%y%m%d'))
    out = OUT_DIR / "results_{}_study_solver_time_mc10.csv".format(datetime.today().strftime('%y%m%d'))
    
    mc10 = benchmark("model_maximum_coverage.py", data, "cplex", "glpk", "gurobi", n=50, save=True, out=out) 
else:
    mc10 = pd.read_csv(r"results/results_231012_study_solver_time_mc10.csv")

mc10.quantile()

In [ ]:
if False:
    data = OUT_DIR / "data_{}_maximum_coverage_25.dat".format(datetime.today().strftime('%y%m%d'))
    out = OUT_DIR / "results_{}_study_solver_time_mc25.csv".format(datetime.today().strftime('%y%m%d'))
    
    mc25 = benchmark("model_maximum_coverage.py", data, "cplex", "glpk", "gurobi", n=50, save=True, out=out) 
else:
    mc25 = pd.read_csv(r"results/results_231012_study_solver_time_mc25.csv")

mc25.quantile()

In [ ]:
if False:
    data = OUT_DIR / "data_{}_maximum_coverage_50.dat".format(datetime.today().strftime('%y%m%d'))
    out = OUT_DIR / "results_{}_study_solver_time_mc50.csv".format(datetime.today().strftime('%y%m%d'))
    
    mc50 = benchmark("model_maximum_coverage.py", data, "cplex", "glpk", "gurobi", n=50, save=True, out=out) 
else:
    mc50 = pd.read_csv(r"results/results_231012_study_solver_time_mc50.csv")

mc50.quantile()

### Summarize results

In [ ]:
from uncertainties import ufloat

entries = [
    ("SC", r"results/results_231012_study_solver_time_sc.csv"),
    ("MC K=10", r"results/results_231012_study_solver_time_mc10.csv"),
    ("MC K=25", r"results/results_231012_study_solver_time_mc25.csv"),
    ("MC K=50", r"results/results_231012_study_solver_time_mc50.csv")
]

with open(OUT_DIR /"results_231012_study_solver_summary.csv", "w+") as f:
            
    for problem, file in entries:
        tmp = pd.read_csv(file)
        line = f"{problem}"
        mean = tmp.mean()
        std = tmp.std()
        for solver in mean.index:
            line += ", {:.2uS}".format(ufloat(mean[solver],1.96*std[solver]))
        f.write(line + '\n')

In [ ]:
fname = OUT_DIR / "results_231012_study_solver_summary.csv"
!type {fname}